# 資料視覺化網站實作2

## Text Mining

> LDA Model：  
Latent Dirichelet Allocation，用以分析字詞的出現頻率，可以進行對文章的主題分類。



> Corpora：  
gensim的class，可以創建提供給LDA Model的Corpus(詞袋)  
可以利用GPT-3.5 API生成文件summary再進行分析。



> 文字處理時建議先進行Preprocessing排除部分泛用字眼(the, and, etc)






In [3]:
#upload key file
from google.colab import files
uploaded = files.upload()

Saving key.txt to key (1).txt


In [4]:
#upload key file(Must named "key.txt")
keyfile = open("/content/key.txt",'r')
TOKEN = keyfile.readline()

openai.api_key = TOKEN

In [ ]:
#upload paper file(Must named "01.txt","02.txt","03.txt")
fileSize = 8 #Define num of file
%cd "/content/paper"
for i in range(fileSize):
  upload = files.upload()



---

以上程式若已經上傳檔案可以跳過

In [1]:
!pip install openai
!pip install pyLDAvis
!pip install stemming

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import gensim
import openai
from pprint import pprint

In [ ]:
#readfile
itemlists = []
for i in range(1,fileSize+1):
  with open('/content/paper/0'+str(i)+'.txt', 'r') as fh:
    tmp = fh.read()
    itemlists.append(tmp.split(','))

In [7]:
# data input function
def chatgptfn(sub_list):
    result = ''
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant"},
            {"role": "user", "content": f"{sub_list} :give me a summary"}
        ]
    )
    for choice in response.choices:
        result += choice.message.content
    return result


In [ ]:
# create corpus
import time
datas = []
for itemlist in itemlists:
  length = int(len(itemlist)/5)
  datas.append([itemlist[0:length], itemlist[length+1:length*2], 
          itemlist[length*2+1:length*3], itemlist[length*3+1:length*4], itemlist[length*4+1:-1]])
for data in datas:
  for i in range(5):
    data[i] = chatgptfn(data[i])
    print(str(i+1),', ',data[i])


至此已將論文進行分段summary，接下來進行預處理：

* Removing stop words: filter out commonly used and auxiliary words (e.g., “the”, “a”, “an”, “in,” “she”)

* Removing irrelevant characters: ignore numbers, punctuation, symbols, etc.

* Tokenization: split a sequence of strings into tokens (e.g., words, keywords, phrases, sentences, and other elements); enables analysis down to the level of segmentation; used in the models, like bag-of-words, for term frequency counting, text clustering, and document matching tasks

* Stemming: reduce inflected words to their root forms (e.g., trouble, troubled, troubling → troubl-); improves the performance of text clustering tasks by reducing dimensions (i.e., the number of terms to be processed)







In [66]:
#Preprocessing texts
from gensim.parsing.preprocessing import remove_stopwords
from stemming.porter2 import stem
import string
texts = []
for data in datas:
  print(data)
  texts.append([stem(word) for word in remove_stopwords(document.lower()).split()] for document in data)
texts = [i for text in texts for i in text]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

['The paper discusses the creation of a new model called GloVe (Global Vectors for Word Representation) for word representations. It analyzes the properties necessary for creating linear meanings in word vectors and argues that global log-bilinear regression models are appropriate for doing so. The paper then proposes a specific weighted least squares model that trains on global word-word co-occurrence counts, making efficient use of statistics while generating a vector space with meaningful substructure. GloVe outperforms other current models on similarity tasks, named entity recognition, and several word analogy tasks.', 'The text discusses a model for learning word vectors that is based on ratios of co-occurrence probabilities. The goal is to distinguish between relevant and irrelevant words in a corpus. The model uses a weighting function in the cost function to give more weight to frequent co-occurrences and less weight to rare ones. The model is compared to other models like skip

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
#create lda model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                    num_topics=3, random_state=100, update_every=1, 
                    chunksize=100, passes=10, alpha='auto', per_word_topics=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
# classify answer
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [72]:
#Store GPT-3.5 result
f = open("/content/result.txt", "a")
for data in datas:
  for item in data:
    f.writelines(item)
    f.writelines('\n\n')
f.close()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
